# Calibrazione della camera

In questo foglio affronteremo uno dei passaggi cardine quando si lavora con una camera: la correzione degli distorsioni!

Bisogna infatti sapere che la maggior parte delle fotocamere introduce una significativa distorsione alle immagini che cattura, le due più importanti sono la distorsione radiale e quella tangenziale. La prima causa alle linee rette di apparire curve e aumenta man mano che ci si allontana dal centro dell'immagine, mentre la seconda è data dal fatto che il piano della lente non è parallelo al piano dell'immagine.

![esempio radiale]("part1/img/calib_radial.jpg")
![esempio tangenziale]("part1/img/tangential_distortion.png")

La procedura che va a calcolare i parametri relativi a questi errori per darci una matrice di correzione è detta **calibrazione intrinseca**

Per questo corso ti servirà anche stampare la scacchiera che troverai nel materiale di questo modulo. Questa risulta uno strumento perfetto in quando ci fornisce dei punti facilmente individuabili e dalle coordinate relative ben definite. Sarà inoltre importante che questa rimanga ben piatta, consigliamo quindi di incollarla ad una superficie solida (ex. cartoncino duro).

Cominciamo a programmare, importiamo innanzitutto le librerie necessarie:

In [ ]:
import cv2
import numpy as np
import yaml

andremo inoltre ad importare una libreria necessaria per l'utilizzo della camera, per chi sta seguendo questo tutorial da raspberry con una picamera, l'importazione sarà la prima, per tutti gli altri la seconda

In [ ]:
from picamera import PiCamera